In [1]:
# Setup
# `jupyter nbextension enable --py --sys-prefix ipyvuetify`

In [2]:
import requests
import json
import pathlib

import plotly.graph_objects as go
import ipyvuetify as v
from ipywidgets import jslink


PTV_API = pathlib.Path("key.txt").read_text()


In [3]:
def get_coordinates(place_query):
    response = requests.get(
        f"https://api.myptv.com/geocoding/v1/locations/by-text?searchText={place_query}&apiKey={PTV_API}"
    )
    location = response.json()["locations"][0]
    return {
        "longitude": location["referencePosition"]["longitude"],
        "latitude": location["referencePosition"]["latitude"],
        "country": location["address"]["countryName"],
        "city": location["address"]["city"],
    }

In [4]:
def get_photovoltaic_data(
    latitude,
    longitude,
    start_year=None, 
    end_year=None
):
    """
    Dokumentation:
    "https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/getting-started-pvgis/api-non-interactive-service_en"
    """
    response = requests.get(
        f"https://re.jrc.ec.europa.eu/api/seriescalc?lat={latitude}&lon={longitude}&horirrad=1&outputformat=json"\
        f"{'&startyear='+str(start_year) if start_year else ''}"\
        f"{'&endyear='+str(end_year) if end_year else ''}"\
    )
    return response.json()["outputs"]["hourly"]
# get_photovoltaic_data(54,13)

In [7]:
def get_data_fig(output):
    return go.Figure(
        data=[
            go.Bar(x=[o["time"] for o in output], y=[o["G(i)"] for o in output])
        ],

        layout_title_text="A Figure Displaying Itself"
    )

# {'time': '20150101:0010', 'G(i)': 0.0, 'H_sun': 0.0, 'T2m': -9.23, 'WS10m': 0.5, 'Int': 0.0}
# G(i): Global irradiance on the inclined plane (plane of the array) (W/m2)
# H_sun: Sun height (degree)
# T2m: 2-m air temperature (degree Celsius)
# WS10m: 10-m total wind speed (m/s)
# Int: 1 means solar radiation values are reconstructed


In [15]:
# Address
address_title = v.Html(
        tag='h1',
        attributes={'title': 'address form'},
        children=['Address']
    )

country = v.TextField(label="Land", v_model="", disabled=True)
city = v.TextField(label="Stadt", v_model="", disabled=True)
latitude = v.TextField(label="Latitude", v_model="", disabled=True)
longitude = v.TextField(label="Longitude", v_model="", disabled=True)
address_row = v.Row(
    children=[
        country,
        city,
        latitude,
        longitude,
    ]
)

search_address_input = v.TextField(label="Adresse", v_model="", prepend_icon="mdi-map-marker")
def get_address(widget, event, data):
    coordinates = get_coordinates(search_address_input.v_model)
    country.v_model = coordinates["country"]
    city.v_model = coordinates["city"]
    latitude.v_model = coordinates["latitude"]
    longitude.v_model = coordinates["longitude"]
    
search_address_btn = v.Btn(children=["such"])
search_address_btn.on_event("click", get_address)
search_address_row=v.Row(
    children=[
        search_address_input, 
        search_address_btn
    ]
)

address_container=v.Container(
    style_='border: 1px solid black;',
    children=[
        v.Container(
            style_="padding=5px;",
            children=[
                address_title,
                address_row,
                search_address_input, 
        search_address_btn
            ])
    ]
)

address_container                
                             
# get photovoltaic data
# data = get_photovoltaic_data(coordinates["latitude"],coordinates["longitude"])
# data_text.v_model = (
#     f'{data}'
# )


Container(children=[Container(children=[Html(attributes={'title': 'address form'}, children=['Address'], tag='…